In [5]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from nltk.stem.porter import *
from gensim.models import Word2Vec,KeyedVectors
from nltk.stem.porter import PorterStemmer

In [6]:
df=pd.read_csv('train.csv')

In [7]:
df.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [8]:
cdf=df.copy()

In [9]:
cdf.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [10]:
cdf.shape

(7274, 3)

In [11]:
cdf.describe()

,tweet_id,sentiment
count,7274.000000,7274.000000
mean,4531.736871,1.299148
std,2617.858745,0.607829
min,2.000000,0.000000
25%,2261.500000,1.000000
50%,4530.500000,1.000000
75%,6796.750000,2.000000
max,9092.000000,3.000000


In [12]:
cdf['sentiment'].value_counts()

1    4311
2    2382
0     456
3     125
Name: sentiment, dtype: int64

In [13]:
tweets=cdf['tweet'].values

Word Tokenization

In [14]:
#nltk.download('punkt')

In [15]:
texts = cdf['tweet'].tolist()

In [16]:
map(word_tokenize, texts)

In [17]:
cdf_lower = cdf['tweet'].str.lower()

In [18]:
cdf_words = cdf_lower.apply(lambda x:word_tokenize(str(x)))

In [19]:
cdf_words

0       [#, sxswnui, #, sxsw, #, apple, defining, lang...
1       [learning, ab, google, doodles, !, all, doodle...
2       [one, of, the, most, in-your-face, ex, ., of, ...
3       [this, iphone, #, sxsw, app, would, b, pretty,...
4       [line, outside, the, apple, store, in, austin,...
5       [#, technews, one, lone, dude, awaits, ipad, 2...
6       [sxsw, tips, ,, prince, ,, npr, videos, ,, toy...
7       [nu, user, rt, @, mention, new, #, ubersocial,...
8       [free, #, sxsw, sampler, on, itunes, {, link, ...
9       [i, think, i, might, go, all, weekend, without...
10      [rt, @, mention, official, #, sxsw, app, û÷sx...
11      [it, 's, official, !, i, 'm, buying, an, ipad,...
12      [they, 're, giving, away, ipad, 2, 's, ,, x, b...
13      [rt, @, mention, we, 're, officially, at, #, s...
14      [#, companies, to, watch, ,, from, the, #, sxs...
15      [rt, @, mention, google, marissa, mayer, ,, fu...
16      [dl, the, #, calyp, app, to, get, into, #, cal...
17      [well,

Initialise Word Lemmatization

In [20]:
all_tweet = cdf[['tweet']]

all_tweet = pd.DataFrame(all_tweet)

all_tweet.columns = ['Tweet']

all_tweet['Tweet'] = all_tweet['Tweet'].str.lower()

In [21]:
all_tweet.head()

,Tweet
0,#sxswnui #sxsw #apple defining language of tou...
1,learning ab google doodles! all doodles should...
2,one of the most in-your-face ex. of stealing t...
3,this iphone #sxsw app would b pretty awesome i...
4,line outside the apple store in austin waiting...


In [22]:
type(all_tweet)

pandas.core.frame.DataFrame

In [23]:


bow_1 = df['tweet'].apply(lambda x:word_tokenize(str(x)))
print(bow_1)

0       [#, sxswnui, #, sxsw, #, apple, defining, lang...
1       [Learning, ab, Google, doodles, !, All, doodle...
2       [one, of, the, most, in-your-face, ex, ., of, ...
3       [This, iPhone, #, SXSW, app, would, b, pretty,...
4       [Line, outside, the, Apple, store, in, Austin,...
5       [#, technews, One, lone, dude, awaits, iPad, 2...
6       [SXSW, Tips, ,, Prince, ,, NPR, Videos, ,, Toy...
7       [NU, user, RT, @, mention, New, #, UberSocial,...
8       [Free, #, SXSW, sampler, on, iTunes, {, link, ...
9       [I, think, I, might, go, all, weekend, without...
10      [RT, @, mention, Official, #, SXSW, App, Û÷SX...
11      [It, 's, official, !, I, 'm, buying, an, iPad,...
12      [They, 're, giving, away, iPad, 2, 's, ,, x, b...
13      [RT, @, mention, We, 're, officially, at, #, S...
14      [#, Companies, to, watch, ,, from, the, #, SXS...
15      [RT, @, mention, Google, Marissa, Mayer, ,, fu...
16      [DL, the, #, Calyp, App, to, get, into, #, Cal...
17      [Well,

In [24]:
bow_2 = []
for i in bow_1:
    bow_2.append(i)
bow_2

[['#',
  'sxswnui',
  '#',
  'sxsw',
  '#',
  'apple',
  'defining',
  'language',
  'of',
  'touch',
  'with',
  'different',
  'dialects',
  'becoming',
  'smaller'],
 ['Learning',
  'ab',
  'Google',
  'doodles',
  '!',
  'All',
  'doodles',
  'should',
  'be',
  'light',
  ',',
  'funny',
  '&',
  'amp',
  ';',
  'innovative',
  ',',
  'with',
  'exceptions',
  'for',
  'significant',
  'occasions',
  '.',
  '#',
  'GoogleDoodle',
  '#',
  'sxsw'],
 ['one',
  'of',
  'the',
  'most',
  'in-your-face',
  'ex',
  '.',
  'of',
  'stealing',
  'the',
  'show',
  'in',
  'yrs',
  'RT',
  '@',
  'mention',
  '&',
  'quot',
  ';',
  'At',
  '#',
  'SXSW',
  ',',
  'Apple',
  'schools',
  'the',
  'mkt',
  'experts',
  '&',
  'quot',
  ';',
  '{',
  'link',
  '}'],
 ['This',
  'iPhone',
  '#',
  'SXSW',
  'app',
  'would',
  'b',
  'pretty',
  'awesome',
  'if',
  'it',
  'did',
  "n't",
  'crash',
  'every',
  '10mins',
  'during',
  'extended',
  'browsing',
  '.',
  '#',
  'Fuckit',
  '

In [25]:

bow_3 = []
def reemovNestings(l): 
    for i in l: 
        if type(i) == list: 
            reemovNestings(i) 
        else: 
            bow_3.append(i)
reemovNestings(bow_2)
bow_3

['#',
 'sxswnui',
 '#',
 'sxsw',
 '#',
 'apple',
 'defining',
 'language',
 'of',
 'touch',
 'with',
 'different',
 'dialects',
 'becoming',
 'smaller',
 'Learning',
 'ab',
 'Google',
 'doodles',
 '!',
 'All',
 'doodles',
 'should',
 'be',
 'light',
 ',',
 'funny',
 '&',
 'amp',
 ';',
 'innovative',
 ',',
 'with',
 'exceptions',
 'for',
 'significant',
 'occasions',
 '.',
 '#',
 'GoogleDoodle',
 '#',
 'sxsw',
 'one',
 'of',
 'the',
 'most',
 'in-your-face',
 'ex',
 '.',
 'of',
 'stealing',
 'the',
 'show',
 'in',
 'yrs',
 'RT',
 '@',
 'mention',
 '&',
 'quot',
 ';',
 'At',
 '#',
 'SXSW',
 ',',
 'Apple',
 'schools',
 'the',
 'mkt',
 'experts',
 '&',
 'quot',
 ';',
 '{',
 'link',
 '}',
 'This',
 'iPhone',
 '#',
 'SXSW',
 'app',
 'would',
 'b',
 'pretty',
 'awesome',
 'if',
 'it',
 'did',
 "n't",
 'crash',
 'every',
 '10mins',
 'during',
 'extended',
 'browsing',
 '.',
 '#',
 'Fuckit',
 '#',
 'Illmakeitwork',
 'Line',
 'outside',
 'the',
 'Apple',
 'store',
 'in',
 'Austin',
 'waiting',
 

In [26]:
bow_final = list(set(bow_3))
bow_final

['spread',
 'anyway',
 'MicroSIM',
 'sink',
 'Yeay',
 'sxsw11',
 'Light',
 'myspace',
 'tmobile',
 'Clover',
 'cle',
 'sgrocks',
 'exercise',
 'Perimeter',
 'Latest',
 'travels',
 'Scarborough',
 'Length',
 '13',
 'Own',
 'pigfucker',
 'pre-',
 'loosing',
 'DuranDuran',
 'populous',
 'Brilliant',
 'sxswi\x89Û\x9d',
 '20+',
 'A',
 'iphone/ipad',
 'Shhh',
 'docs',
 'Music',
 'redirect',
 'retailers',
 'Grounlink',
 'sauce',
 'due',
 'Initial',
 'wiiings',
 'san',
 "'S",
 'bringing',
 'respect',
 'autocorrected',
 'Giant',
 'Know',
 'mhealth',
 'chat/text',
 'Omar',
 'hire',
 'bookbook',
 'Brain',
 'RecycleMatch',
 'Enuf',
 'Kit',
 'Notes',
 'Ze',
 'mar',
 'alone',
 'Holy',
 'southpaw',
 '420',
 'shipments',
 'neuf',
 'boots',
 'jcpenney',
 'lease',
 'Facebook',
 'mini-store',
 'hint',
 'Rt',
 'enjoys',
 'knowyouraudience',
 'moma',
 'shortly',
 'Cafe',
 'His',
 'mad',
 'optional',
 'overnight',
 'BPM',
 'ex',
 'AICN',
 'you.',
 'river',
 'expectation',
 'futureoftouch',
 'Louisiana',
 'h

In [27]:
len(bow_final)

11938

In [28]:
len(bow_3)

177158

In [32]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [33]:

bow_5 = df['tweet'].apply(lambda x:lemmatizer.lemmatize(str(x)))
print(bow_5)

0       #sxswnui #sxsw #apple defining language of tou...
1       Learning ab Google doodles! All doodles should...
2       one of the most in-your-face ex. of stealing t...
3       This iPhone #SXSW app would b pretty awesome i...
4       Line outside the Apple store in Austin waiting...
5       #technews One lone dude awaits iPad 2 at Apple...
6       SXSW Tips, Prince, NPR Videos, Toy Shopping Wi...
7       NU user RT @mention New #UberSocial for #iPhon...
8          Free #SXSW sampler on iTunes {link} #FreeMusic
9       I think I might go all weekend without seeing ...
10      RT @mention Official #SXSW App Û÷SXSW GOÛª b...
11      It's official! I'm buying an iPad. #SXSW #elevate
12      They're giving away iPad 2's, x boxes and book...
13      RT @mention We're officially at #SXSW! Come by...
14      #Companies to watch, from the #SXSW trade show...
15      RT @mention Google Marissa Mayer, future of lo...
16      DL the #Calyp App to get into #Calyp Casa at #...
17      Well y

In [34]:
ps=PorterStemmer()

In [35]:
bow_6 = df['tweet'].apply(lambda x:ps.stem(str(x)))
print(bow_6)

0       #sxswnui #sxsw #apple defining language of tou...
1       learning ab google doodles! all doodles should...
2       one of the most in-your-face ex. of stealing t...
3       this iphone #sxsw app would b pretty awesome i...
4       line outside the apple store in austin waiting...
5       #technews one lone dude awaits ipad 2 at apple...
6       sxsw tips, prince, npr videos, toy shopping wi...
7       nu user rt @mention new #ubersocial for #iphon...
8            free #sxsw sampler on itunes {link} #freemus
9       i think i might go all weekend without seeing ...
10      rt @mention official #sxsw app û÷sxsw goûª b...
11         it's official! i'm buying an ipad. #sxsw #elev
12      they're giving away ipad 2's, x boxes and book...
13      rt @mention we're officially at #sxsw! come by...
14      #companies to watch, from the #sxsw trade show...
15      rt @mention google marissa mayer, future of lo...
16      dl the #calyp app to get into #calyp casa at #...
17      well y

In [36]:
without_special_char=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]', '', x), bow_3)))

In [37]:
without_special_char

['sxswnui',
 'sxsw',
 'apple',
 'defining',
 'language',
 'of',
 'touch',
 'with',
 'different',
 'dialects',
 'becoming',
 'smaller',
 'Learning',
 'ab',
 'Google',
 'doodles',
 'All',
 'doodles',
 'should',
 'be',
 'light',
 'funny',
 'amp',
 'innovative',
 'with',
 'exceptions',
 'for',
 'significant',
 'occasions',
 'GoogleDoodle',
 'sxsw',
 'one',
 'of',
 'the',
 'most',
 'inyourface',
 'ex',
 'of',
 'stealing',
 'the',
 'show',
 'in',
 'yrs',
 'RT',
 'mention',
 'quot',
 'At',
 'SXSW',
 'Apple',
 'schools',
 'the',
 'mkt',
 'experts',
 'quot',
 'link',
 'This',
 'iPhone',
 'SXSW',
 'app',
 'would',
 'b',
 'pretty',
 'awesome',
 'if',
 'it',
 'did',
 'nt',
 'crash',
 'every',
 'mins',
 'during',
 'extended',
 'browsing',
 'Fuckit',
 'Illmakeitwork',
 'Line',
 'outside',
 'the',
 'Apple',
 'store',
 'in',
 'Austin',
 'waiting',
 'for',
 'the',
 'new',
 'iPad',
 'SXSW',
 'link',
 'technews',
 'One',
 'lone',
 'dude',
 'awaits',
 'iPad',
 'at',
 'Apples',
 'SXSW',
 'store',
 'link',


In [38]:
len(without_special_char)

129308

In [39]:
type(without_special_char)

list

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(without_special_char)

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

independent
{'independently', 'independent', 'independence', 'independents'}
right
{'fight', 'wight', 'light', 'alight', 'might', 'eight', 'aught', 'sight', 'right', 'tight', 'night', 'aright', 'bight'}
space
{'espace', 'space'}
voodoo
{'boohoo', 'hoodoo', 'voodoo', 'ooooh', 'wooton'}
mobile
{'mobile'}
bradford
{'b-form', 'bamford', 'bradford', 'bedford', 'basford', 'radford', 'barford', 'landform'}
overweening
{'overweening'}
fitting
{'fixing', 'fining', 'firing', 'siting', 'citing', 'filing', 'biting', 'fitting'}
autocorrect
{'autocorrect'}
everybody
{'everybody'}
foxton
{'foxton'}
parties
{'parted', 'parried', 'parties', 'partie'}
rocks
{'frocks', 'smocks', 'rocks', 'stocks', 'socks', 'crocks', 'shocks'}
dustmen
{'juster', 'dustmen', 'justtext'}
was
{'nam', 'lam', 'war', 'pam', 'wat', 'wan', 'yam', 'dam', 'jam', 'ywam', 'bam', 'am', 'mam', 'cam', 'fam', 'wal', 'wim', 'swam', 'wax', 'sam', 'wham', 'wa', 'zam', 'ram', 'wai', 'wag', 'ham', 'wm', 'tam', 'wom', 'wcm', 'w1m', 'wav', 'wah'

photobooth
{'photobooth'}
weathering
{'feathering', 'tethering', 'weathering'}
socialnews
{'socialnews'}
chilltab
{'chilltab'}
wasteoftimeatsxsw
{'wasteoftimeatsxsw'}
olderadults
{'olderadults'}
rockaroke
{'rockaroke'}
swissmade
{'swissmade'}
going
{'boing', 'booing', 'oozing', 'toing', 'wooing', 'owing', 'cooing', 'doing', 'going'}
internet
{'internees', 'internet'}
bemyneighbor
{'bemyneighbor'}
her
{'leu', 'elr', 'l/r', 'lear', 'lor', 'lsr', 'ter', 'ley', 'l-r', 'cer', 'les', 'lur', 'lea', 'ber', 'lew', 'wer', 'let', 'le', 'er', 'led', 'per', 'fer', 'der', 'ger', 'lar', 'len', 'lek', 'mer', 'leer', 'her', 'lex', 'lec', 'lev', 'ver', 'lner', 'ner', 'lr', 'yer', 'lei', 'ker', 'ser', 'aer', 'leo', 'lep', 'leg', 'ltr', 'ldr', 'lee', 'lcr'}
jolla
{'jools', 'jolla', 'joolz', 'hoopla', 'romola'}
popupshop
{'popupshop'}
self-respecting
{'self-respecting'}
getting
{'getting'}
perspective
{'respective', 'prospective', 'perspective'}
blahzay
{'blahzay'}
hollergram
{'hollergram'}
popupstores
{'p

gadgetenvy
{'gadgetenvy'}
crowdsourcing
{'crowdsourcing'}
thecivilwars
{'thecivilwars'}
fbi
{'f.i', 'fy', 'fi', 'fei', 'fmi', 'fym', 'kyi', 'fyn', 'fyt', 'fdi', 'yi', 'fai', 'fki', 'fpi', 'fri', 'fii', 'fbi'}
slr
{'dsr', 'slr'}
indieauthors
{'indieauthors'}
heavenlymichael
{'heavenlymichael'}
transitapps
{'transitapps'}
jeremy
{'jeremiah', 'jeremy', 'jerome', 'ieremia', 'jeremias', 'yeremi'}
rumour
{'rumors', 'rulor', 'tumor', 'rumour', 'humor'}
outmarkets
{'outmarkets'}
soft
{'sq ft', 'sift', 'sq.ft', 'qft', 'soft'}
authenticationdesign
{'authenticationdesign'}
his
{'als', 'hlh', 'ols', 'hes', 'pls', 'hl', 'nls', 'hvs', 'has', 'ls', 'els', 'hbs', 'hns', 'h2s', 'hals', 'mls', 'hds', 'sls', 'hs', 'hss', 'tls', 'cls', 'gls', 'hfs', 'his', 'hms', 'hols', 'hcs', 'hrs', 'rls', 'ils', 'hla', 'hus', 'lhs'}
hit
{'ott', 'ctt', 'dtt', 'hot', 'hat', 'ntt', 'het', 'hut', 'itt', 'tt', 'hte', 'hta', 'att', 'ztt', 'hst', 'htv', 'hutt', 'ptt', 'ht', 'hit', 'hatt', 'hrt', 'ttt'}
convert
{'convert', 'ca

networkcircles
{'networkcircles'}
lucy
{'lucy', 'lucas'}
asked
{'aske', 'asks', 'asked', 'ask'}
greatergood
{'greatergood'}
uses
{'usdaw', 'ushaw', 'ussr', 'uses', 'usgs', 'uss'}
both
{'bets', 'bogs', 'bos', 'boys', 'bts', 'boks', 'blots', 'both', 'boats', 'bats', 'boss', 'tots', 'boas', 'bott', 'bouts', 'boots', 'boos', 'ots', 'bits', 'dots', 'mots', 'boto', 'bot', 'bows', 'buts', 'bois', 'bons', 'nots', 'cots', 'hots', 'lots', 'bods', 'bobs', 'bolts', 'botes', 'rots', 'pots'}
crowdbeacon
{'crowdbeacon'}
sponsor
{'sponsor'}
sxswnui
{'sxswnui'}
profit
{'photic', 'prolix', 'protek', 'profit', 'protein', 'proto', 'portia', 'pradip', 'erotic', 'pratap', 'prop', 'protz', 'proton', 'probit'}
belgium
{'belgium'}
melissa
{'nerissa', 'melissa', 'marissa'}
unofficial
{'unofficial'}
restroomi
{'restroomi'}
wakeuplaughing
{'wakeuplaughing'}
geekspringbreak
{'geekspringbreak'}
showcase
{'showcases', 'showcase'}
grouptexting
{'grouptexting'}
mussini
{'mussini'}
apps
{'adapts', 'epps', 'lapps', 'app

{'annunciata'}
index
{'indo', 'inge', 'ide', 'hinde', 'index', 'inder', 'indie', 'linde', 'ine', 'ince', 'ind', 'indy'}
hustling
{'hustling'}
dig
{'divi', 'dii', 'digit', 'dini', 'digs', 'gigi', 'diii', 'didi', 'tigi', 'dig', 'dili'}
start-ups
{'start-ups', 'startup'}
reckon
{'brecon', 'reckon', 'recor', 'redon'}
turnstones
{'turnstones'}
grandad
{'grandad'}
partylikeits
{'partylikeits'}
safety
{'salute', 'sainte', 'spite', 'capet', 'papeete', 'pete', 'sante', 'spate', 'speke', 'sapt', 'gamete', 'saute', 'capote', 'safety'}
endeavour
{'endeavour'}
duration
{'duration'}
marketer
{'marketer'}
mobilesearch
{'mobilesearch'}
types
{'typo', 'topos', 'types', 'tyros'}
lifelinetotheworld
{'lifelinetotheworld'}
vectorbased
{'vectorbased'}
freecreditscore
{'freecreditscore'}
essdub
{'essdub'}
jobs
{'jobson', 'iosco', 'jobs'}
spastics
{'spastics'}
pentagram
{'pentagram'}
booth
{'booths', 'bootie', 'booth', 'bootle', 'soothe'}
itashimasu
{'itashimasu'}
hardcore
{'hardcore'}
cbatsxsw
{'cbatsxsw'}
d

yourselvesseems
{'yourselvesseems'}
garvey
{'garvie', 'gartree', 'garvey'}
ipadmadness
{'ipadmadness'}
cocom
{'cocom'}
sxswk
{'sxswk'}
playeripod
{'playeripod'}
justsayin
{'justsayin'}
thedaily
{'thedaily'}
propping
{'propping'}
oldschool
{'oldschool'}
molla
{'gorilla', 'godzilla', 'molla', 'maxilla', 'mohill', 'marilla', 'milla', 'melilla', 'moila', 'bonilla', 'manilla'}
maudie
{'maudie'}
sxswbusy
{'sxswbusy'}
popping
{'doolin', 'popping', 'pontin', 'chopin', 'pepin', 'pooling', 'pippin', 'poppins', 'poop', 'pogodin', 'looping', 'jospin', 'papin', 'poplin', 'pooper'}
apps
{'paphos', 'zippo', 'zapp', 'campos', 'zappa', 'zapper', 'nappes', 'hippos', 'zapped', 'lapps', 'apps'}
mktgmoment
{'mktgmoment'}
thingsthatdontgotogether
{'thingsthatdontgotogether'}
deja vu
{'deja vu'}
caniexpenseanipad
{'caniexpenseanipad'}
rewardstype
{'rewardstype'}
iphoneography
{'iphoneography'}
nitrate
{'citrate', 'nitrate'}
digitalis
{'digitalis'}
needs
{'neds', 'nerd', 'nerys', 'needs', 'herds'}
stoned
{'in

livesteam
{'livesteam'}
meatpointer
{'meatpointer'}
weinschenk
{'weinschenk'}
post
{'kostic', 'postal', 'post-', 'posts', 'poetic', 'poster', 'post', 'posted', 'nostic', 'posc', 'softpc'}
soon
{'solow', 'soton', 'soho', 'spoor', 'spoon', 'boohoo', 'stood', 'spool', 'stook', 'stool', 'soco', 'stoop', 'slovo', 'soo', 'soot', 'shoom', 'soglo', 'shoo', 'sooty', 'sooth', 'spook', 'sloop', 'soto', 'solon', 'snoop', 'scoot', 'socio', 'solos', 'hoodoo', 'ooooh', 'soros', 'so on', 'scoop', 'ooo', 'sodom', 'spoof', 'so-so', 'solo', 'shook', 'shoot', 'sokoto', 'soon', 'sobor', 'oooh', 'shoob', 'voodoo', 'souto', 'snook', 'swoop', 'sogo', 'swoon'}
white
{'ice-white', 'rewrite', 'white'}
audioboo
{'audioboo'}
suasxsw
{'suasxsw'}
zazzlsxsw
{'zazzlsxsw'}
urbanmyth
{'urbanmyth'}
california
{'california'}
announced
{'announced'}
restraints
{'restraints'}
bankinnovation
{'bankinnovation'}
macchio
{'macchio'}
winssxsw
{'winssxsw'}
needy
{'needy', 'neddy', 'gerdy', 'nerd', 'nervy'}
going
{'fing', 'boing',

{'shuffleboard'}
bankinnovate
{'bankinnovate'}
notepaper
{'notepaper'}
phonecamera
{'phonecamera'}
hackathon
{'hackathon'}
grind
{'grinds', 'grind', 'grinder'}
ratingrecommendation
{'ratingrecommendation'}
suggestionskind
{'suggestionskind'}
everywhere
{'everywhere'}
edgar
{'edgar', 'eduard'}
bushwhacked
{'bushwhacked'}
ago
{'argo', 'ago', 'zygo'}
your
{'yours', 'yore', 'yourn', 'your', 'toure', 'youse'}
david
{'vcid', 'rancid', 'david', 'avoid', 'bacci', 'avid', 'acid'}
finney
{'binney', 'linley', 'finney', 'pinney', 'linnet', 'linzey'}
outcast
{'outcast', 'outlast'}
adloopz
{'adloopz'}
appdownloading
{'appdownloading'}
divisions
{'envisioned', 'excisions', 'divisions', 'incisions', 'elisions', 'invasions', 'revisions', 'evasions', 'visions', 'environs'}
crowdsourced
{'crowdsourced'}
plc
{'ple', 'pla', 'pl.', 'plm', 'plo', 'pls', 'plc', 'pl', 'plr', 'pln', 'paz', 'plf', 'pld', 'plp', 'ply', 'plv', 'piz', 'lz', 'pli'}
eston
{'stoy', 'esto', 'eston'}
in that
{'in that'}
powerboat
{'powe

bitlyhtdfim
{'bitlyhtdfim'}
whispergram
{'whispergram'}
sarah
{'haan', 'farah', 'rajah', 'haigh', 'bahai', 'hamad', 'hagan', 'haas', 'hadar', 'havas', 'salah', 'ayah', 'naha', 'hash', 'rahal', 'halam', 'hata', 'hagar', 'kahan', 'aah', 'hah', 'haag', 'ahh', 'maha', 'hajar', 'fatah', 'sarah', 'harsh', 'harar', 'hana', 'cahal', 'aaah', 'taha', 'mahal', 'hayat', 'aaaah', 'haya', 'hahn', 'wahab', 'habash', 'hatch', 'aha', 'ahab', 'rafah', 'amah', 'sabah', 'fahan', 'hath', 'hama', 'halal', 'haran', 'hamas', 'ahhhh', 'hannah', 'ha-ha', 'naaah', 'ahhh', 'hanan', 'hasan', 'shah', 'varah', 'hamzah', 'rahab', 'mahaz', 'asah'}
di
{'dti', 'mmi', 'dmz', 'dci', 'dei', 'dma', 'mi', 'dmk', 'imi', 'dli', 'dm3', 'dvi', 'omi', 'tmi', 'demi', 'emi', 'dai', 'dmu', 'dm1', 'rmi', 'doi', 'dmr', 'dmj', 'dii', 'dri', 'dm', 'dml', 'dim', 'ddi', 'bmi', 'dme', 'dms', 'fmi', 'ami', 'dmt', 'hmi', 'dpi', 'cmi', 'di'}
six
{'sax', 'sex', 'smg', 'sms', 'six', 'sm3', 'sx', 'smr', 'sox', 'sfx', 'sma', 'sml', 'smc', 'smt', 

In [ ]:
from textblob import TextBlob 

In [ ]:
a = without_special_char          # incorrect spelling 
print("original text: "+(a)) 
  
b = TextBlob(a) 
  
# prints the corrected spelling 
print("corrected text: "+(b.correct()))